# Executive summary

I use the jargon $S, T, U, V$ from `companion-solver-implementation-reference.pdf`

1. something spooky: quadratic formula seems to work 54% of the time, while one of the coefficients is > 0 54% of the time. 
2. ok beyond the 54% coincidence, it's weird that the quadratic formula isn't working
3. $S$ and $T$ are strongly anticorrelated. This could be bad for linear algebra.
4. $U$ and $V$ are `-0.434316` anticorrelated, thank god.

In [10]:
from functools import reduce, partial
product = partial(reduce, lambda x,y: x * y)

import jax.numpy as jnp
from jax import random, vmap, grad

import arviz as az
from matplotlib import pyplot as plt
from ipywidgets.widgets import FloatSlider
from ipywidgets import interact
from scipy.interpolate import BSpline
from scipy.stats import gaussian_kde, zscore

import numpyro
import numpyro.distributions as dist
import numpyro.optim as optim
from numpyro.diagnostics import hpdi, print_summary
from numpyro.infer import Predictive, SVI, Trace_ELBO, init_to_value, NUTS, MCMC
from numpyro.infer.autoguide import AutoLaplaceApproximation, AutoNormal, AutoDiagonalNormal

import xgboost as xgb

import pandas as pd

from report import Report

key = random.PRNGKey(0)

In [11]:

reportD = Report("data/naive-newton-1026/d.csv", abso=False, unknown_feature="D")
reportB0 = Report("data/naive-newton-1026/b0.csv", abso=False, unknown_feature="B0")
reportB1 = Report("data/naive-newton-1026/b1.csv", abso=False, unknown_feature="B1")
reportB2 = Report("data/naive-newton-1026/b2.csv", abso=False, unknown_feature="B2")


In [12]:
# doing something where the root doesn't matter
df = pd.concat((reportD.df, reportB0.df, reportB1.df, reportB2.df)).drop(["Root", "newton_delta", "squared_newton_delta", "Y"], axis=1)

In [13]:
# see companion-solver-implementation-reference.pdf
# I_k = xk^2 + V x + U
def U_0(A, D, B1, B2):
    return - D ** (3 + 1) / A / 3 ** (2 * 3) / B1 / B2
def U_1(A, D, B0, B2): 
    return - D ** (3 + 1) / A / 3 ** (2 * 3) / B0 / B2
def U_2(A, D, B0, B1): 
    return - D ** (3 + 1) / A / 3 ** (2 * 3) / B0 / B1

def V_0(A, D, B1, B2): 
    return B1 + B2 + (1 / A / 3 ** 3 - 1) * D
def V_1(A, D, B0, B2): 
    return B0 + B2 + (1 / A / 3 ** 3 - 1) * D
def V_2(A, D, B0, B1): 
    return B0 + B1 + (1 / A / 3 ** 3 - 1) * D

In [14]:
(U_0(85, df.D, df.B1, df.B2) > 0).sum() / df.shape[0], (U_1(85, df.D, df.B0, df.B2) > 0).sum() / df.shape[0], (U_2(85, df.D, df.B0, df.B1) > 0).sum() / df.shape[0]

(0.0, 0.0, 0.0)

In [15]:
# V is positive 50% of the time, well when you make `A` variable drawn from the data they each go exactly to 54 which is weird.
(V_0(df.A, df.D, df.B1, df.B2) > 0).sum() / df.shape[0], (V_1(df.A, df.D, df.B0, df.B2) > 0).sum() / df.shape[0], (V_2(df.A, df.D, df.B0, df.B1) > 0).sum() / df.shape[0]

(0.5464, 0.542625, 0.544925)

In [16]:
# quadratic formula
def discriminant(V, U): 
    return V ** 2 - 4 * 1 * U
def left(V): 
    return - V / 2 / 1

def left_root(V, U): 
    return left(V) - discriminant(V, U) ** (1 / 2) / 2
def right_root(V, U): 
    return left(V) + discriminant(V, U) ** (1 / 2) / 2

left_root(V_0(85, df.D, df.B1, df.B2), U_0(85, df.D, df.B1, df.B2)).mean(), right_root(V_0(85, df.D, df.B1, df.B2), U_0(85, df.D, df.B1, df.B2)).mean()

(-75082024999.88412, 74462246769.95874)

In [26]:
V = V_0(df.A, df.D, df.B1, df.B2)
U = U_0(df.A, df.D, df.B1, df.B2)
root = right_root(V, U)
df = df.assign(V_0=V, U_0=U)
(df.assign(should_be_zero=root ** 2 + V * root + U)[df.V_0 < 0].should_be_zero < 1e-7).sum() / df[df.V_0 < 1e-7].shape[0] #  < 1e-7).sum() / df.shape[0]# :( should be 100

0.5597442680776014

In [ ]:
df.assign(should_be_zero=root ** 2 + V * root + U)

In [34]:
(V ** 2 - 4 * U).mean()

7.676034463710412e+23

In [18]:
# I_D = D ^ {n + 1} + T * D + S
def S(A, *balances): 
    n = len(balances)
    return - A * n ** (2 * n) * product(balances) * sum(balances)

def T(A, *balances): 
    n = len(balances)
    return (A + 1 / n ** n) * n ** (2 * n) * product(balances)

In [19]:
S_A85 = S(85, df.B0, df.B1, df.B2)
T_A85 = T(85, df.B0, df.B1, df.B2)
df = df.assign(S=S_A85, T=T_A85)
downsampled = df.sample(frac=0.1)
df[["S", "T"]].corr()

,S,T
S,1.000000,-0.980513
T,-0.980513,1.000000


In [28]:
(df["T"] > 0).sum() / df.shape[0], (df["S"] < 0).sum() / df.shape[0]

(1.0, 1.0)

In [ ]:
df[["V_0", "U_0"]].corr()

In [ ]:
(discriminant(V, U) < 1e-3).sum() # complex numbers are not the reason I broke the quadratic formula. But it's nice to see that 

In [ ]:
discriminant(V, U)

In [145]:
df.B2.mean()

115039925432.79308